In [3]:
import pandas as pd
import json
import requests
import io
import h5py


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from google.colab.patches import cv2_imshow

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1LcKqf1d7bctw5lx0YZf31kCUF0zEYOsi'
downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('MNIST_synthetic.zip')


from zipfile import ZipFile
file_name = "MNIST_synthetic.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 
dataset = h5py.File('MNIST_synthetic.h5', 'r')


#winnie
print(dataset['test_dataset'].shape)
print(dataset['train_dataset'].shape)
print(dataset['train_labels'].shape)
# number of images = 56000
# height and width of each image =  64
# number of color channel = 1
# so each image is 64*64*1
X_test_0=dataset['test_dataset'].value
X_train_0=dataset['train_dataset'].value
y_train_0=dataset['train_labels'].value

File Name                                             Modified             Size
MNIST_synthetic.h5                             2020-11-21 00:57:46    287842048
Extracting all the files now...
Done!
(14000, 64, 64, 1)
(56000, 64, 64, 1)
(56000, 5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


In [4]:
# winnie
import cv2 
import numpy as np
# the first image  
img=X_train_0[0]
################################################################################
# Function1: get image contour

def get_signal(img,index):
  length=0
  signal=-1
  for j in img[:,index]:
      if j[0]>0:
          length=length+1
          if length>7:
              signal=1
              break
  return signal


def get_contour_new(img):
  standard=np.zeros((64,1))
  cropped_scale=[]
  x1_x2_len=[]
  x=[]
  ptr=0
  acc=0
  for i in range(64):
    if [False] not in (img[:,i]==standard):
      x.append(i)
     
  for i in range(len(x)-1):
    if x[i]+1 != x[i+1]:

      if x[i+1]-x[i]==3:
        for j in range(2):
          if get_signal(img,x[i]+j)==1:
            x1,x2,y1,y2=x[i]-1,x[i+1]+1,25,38
            cropped_scale.append([x1,x2,y1,y2])
            break
      if x[i+1]-x[i]==4:
        for j in range(3):
          if get_signal(img,x[i]+j)==1:
            x1,x2,y1,y2=x[i]-1,x[i+1]+1,25,38
            cropped_scale.append([x1,x2,y1,y2])
            break


            
      elif x[i+1]-x[i]>3:
          x1,x2,y1,y2=x[i]-1,x[i+1]+1,25,38
          cropped_scale.append([x1,x2,y1,y2])

  if len(cropped_scale)>5:
      #x1_x2_len=[cropped_scale[i][1]-cropped_scale[i][0]  for i in range(len(cropped_scale))]
      #cropped_scale_new= cropped_scale.sort(key=(lambda x: x[0]-x[1]) )
      #cropped_scale_new.sort()
      cropped_scale.sort(key=(lambda x: x[0]-x[1]) )
      cropped_scale=cropped_scale[0:5]

  cropped_scale.sort()
  return cropped_scale


    




################################################################################
 # Function3: show cropped image
def show_cropped_image(img,cropped_scale):
  cv2_imshow(img)
  for i in range(len(cropped_scale)):
    x1, x2, y1, y2 = cropped_scale[i][0],cropped_scale[i][1],cropped_scale[i][2],cropped_scale[i][3]
    cropped = img[y1:y2, x1:x2]
    # function showing the image
    cv2_imshow(cropped)


################################################################################
# Function 5: new image resize
def resize(x1,x2,y1,y2,img,w,h):
  
  image=np.zeros((0,w))
  h0=(h-(y2-y1))
  w0=(w-(x2-x1))
  #print(h0)
  #print(img.shape)
  if h0%2!=0:
    h_diff=int(float(h0)/2)
    a=np.zeros((h_diff,w))
    image=np.append(image,np.zeros((h_diff,w)),axis=0)
    if w0%2 !=0:
      w_diff=int(float(w0)/2)
      #print(np.zeros((y2-y1,w_diff)).shape)
      #print(img[y1:y2, x1:x2].shape)
      tmp=np.append(np.zeros((y2-y1,w_diff)),img[y1:y2, x1:x2],axis=1)
      #print(tmp.shape,np.zeros((y2-y1,w_diff+1)).shape)
      tmp=np.append(tmp,np.zeros((y2-y1,w_diff+1)),axis=1 )
      #print(tmp.shape)
      image=np.append(image,tmp,axis=0)
      image=np.append(image,np.zeros((h_diff+1,w)),axis=0)
    else:
      w_diff=int(w0/2)
      tmp=np.append(np.zeros((y2-y1,w_diff)),img[y1:y2, x1:x2],axis=1)
      tmp=np.append(tmp,np.zeros((y2-y1,w_diff)),axis=1 )
      image=np.append(image,tmp,axis=0)
      image=np.append(image,np.zeros((h_diff+1,w)),axis=0)

  else:
    h_diff=int(h0/2)
    image=np.append(image,np.zeros((h_diff,w)),axis=0)
    
    if w0%2 !=0:
    
      w_diff=int(float(w0)/2)
      tmp=np.append(np.zeros((y2-y1,w_diff)),img[y1:y2, x1:x2],axis=1)
      tmp=np.append(tmp,np.zeros((y2-y1,w_diff+1)),axis=1 )
      image=np.append(image,tmp,axis=0)
      image=np.append(image,np.zeros((h_diff,w)),axis=0)
    else:
      w_diff=int(w0/2)
      tmp=np.append(np.zeros((y2-y1,w_diff)),img[y1:y2, x1:x2],axis=1)
      tmp=np.append(tmp,np.zeros((y2-y1,w_diff)),axis=1 )
      image=np.append(image,tmp,axis=0)
      image=np.append(image,np.zeros((h_diff,w)),axis=0)
  return image



def image_resize_new(img,cropped_scale,w,h):
  tmp=np.zeros((64,64))
  for i in range(64):
    for j in range(64):
      tmp[i][j]=img[i][j][0]

  image=np.zeros((5,w,h))
  dim = (w, h)
  ptr=0
  for i in range(len(cropped_scale)):
    x1, x2, y1, y2 = cropped_scale[i][0],cropped_scale[i][1],cropped_scale[i][2],cropped_scale[i][3]
    if x2-x1>16:
      diff=int((x2-x1)/2)
      resize_image=resize(x1,x2-diff,y1,y2,tmp,w,h)
      image[ptr]=resize_image
      cv2_imshow(resize_image)
      ptr=ptr+1
      resize_image=resize(x2-diff,x2,y1,y2,tmp,w,h)
      image[ptr]=resize_image
      ptr=ptr+1
      cv2_imshow(resize_image)

    else:
      resize_image=resize(x1,x2,y1,y2,tmp,w,h)
      image[ptr]=resize_image
      ptr=ptr+1
    #cv2_imshow(resize_image)
  return image

################################################################################
# Function 6: rearrange the image
def training_resize(X_train_0,w,h,size):
  X_train_new=np.zeros((size,w,h))
  ptr=0
  for i in range(len(X_train_0)):
    if i%1000==0:
      print('iteration:',i)
    img=X_train_0[i]
    #cv2_imshow(img)
    #contours=get_contour(img)
    #cropped_scale=cropped_image(contours,img)
    cropped_scale=get_contour_new(img)
    image=image_resize_new(img, cropped_scale,w,h)
    #X_train_new=np.append(X_train_new,image,axis=0)
    for j in range(5):
      X_train_new[ptr]=image[j]
      ptr=ptr+1
  return X_train_new



In [5]:
# training dataset
# X_train_0 ==> X_train_1
X_train_1=training_resize(X_train_0[0:56000],16,16,280000)
# testing dataset
# X_test_0 ==> X_test_1
X_test_1=training_resize(X_test_0[0:14000],16,16,70000)






iteration: 0


iteration: 1000


iteration: 2000


iteration: 3000


iteration: 4000


...

iteration: 9000


iteration: 10000


iteration: 11000


iteration: 12000


iteration: 13000


In [6]:
length=len(y_train_0)
print(length)
a=np.zeros((length*5,1))
for i in range(length):
  a[5*i][0]=y_train_0[i][0]
  a[5*i+1][0]=y_train_0[i][1]
  a[5*i+2][0]=y_train_0[i][2]
  a[5*i+3][0]=y_train_0[i][3]
  a[5*i+4][0]=y_train_0[i][4]
print(a.shape)

56000
(280000, 1)


In [7]:
import argparse
import os

import numpy as np
#from PIL import Image
#from scipy.misc import imresize
from scipy.ndimage.interpolation import rotate
from scipy.ndimage import shift
'''
def random_rotation(image, angle_range=(0, 180)):
    h, w, _ = image.shape
    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h, w))
    return image
'''
n=2000
ptr=0
indexs=np.random.randint(280000, size=n)
print(indexs)
X_train_1_rotate=np.zeros((n,16,16))
y_train_1_rotate=np.zeros((n,1))
for index in indexs:
  if ptr%10000==0:
    print(ptr)
  degree=np.random.uniform(-18.0,18.0)
  #h=np.random.uniform(-1.0,1.0)
  #w=np.random.uniform(-1.0,1.0)
  #cv2_imshow(X_train_1[i])
  #tmp=shift(X_train_1[i],shift=(w,h))
  #cv2_imshow(tmp)
  X_train_1_rotate[ptr]=rotate(X_train_1[index],angle=degree,reshape=False)
  #cv2_imshow(X_train_1_rotate[i])
  y_train_1_rotate[ptr]=a[index]
  ptr=ptr+1

X_train_1_new=np.append(X_train_1,X_train_1_rotate,axis=0)
y_train_1_new=np.append(a,y_train_1_rotate,axis=0)

print(X_train_1_new.shape)
print(y_train_1_new.shape)

tmp=np.append(X_train_1_new,X_test_1,axis=0)
mean=tmp.mean()
std=tmp.std()

X_train_2=(X_train_1_new-tmp.mean())/std
X_test_2=(X_test_1-tmp.mean())/std

[127055 147443 102314 ... 136907  33143 206999]
0
(282000, 16, 16)
(282000, 1)


In [8]:
import numpy as np
print(X_train_2.shape)
X_train=np.zeros((len(X_train_2),1,16,16))
for i in range(len(X_train)):
  X_train[i][0]=X_train_2[i]
print(X_train.shape)

tmp_X=np.append(X_train[0:260000],X_train[280000:282000],axis=0)
tmp_X=np.append(tmp_X,X_train[260000:280000],axis=0)
print(tmp_X.shape)
X_train=torch.from_numpy(tmp_X)

tmp_y=np.append(y_train_1_new[0:260000],y_train_1_new[280000:282000],axis=0)
tmp_y=np.append(tmp_y,y_train_1_new[260000:280000],axis=0)
print(tmp_y.shape)
y_train=torch.from_numpy(tmp_y)


import numpy as np
print(X_test_2.shape)
X_test=np.zeros((len(X_test_2),1,16,16))
for i in range(len(X_test)):
  X_test[i][0]=X_test_2[i]
X_test.shape

X_test=torch.from_numpy(X_test)

(282000, 16, 16)
(282000, 1, 16, 16)
(282000, 1, 16, 16)
(282000, 1)
(70000, 16, 16)


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

!pip install pycuda
import torch 
import pycuda.driver as cuda
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda.init()
torch.cuda.current_device()
print(cuda.Device(0).name())
#net.to(device)

torch.cuda.empty_cache()

     |████████████████████████████████| 1.6MB 12.3MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621418 sha256=06e08ed8b5663e891e91b13996aa5ee1eb613f561e71c606d717d058d59775eb
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.4-py2.py3-none-any.whl size=59110 sha256=fb0404362325d5ad95c9ee8647632c7b3698540808ce2ec5695e711cd9e594d9
  Stored in directory: /root/.cache/pip/wheels/6f/2a/6e/0b210d900f2b6caca133395d2fa153d8a9c03ad42c3c5d55bf
Successfully built pycuda pytools
Tesla V100-SXM2-16GB


In [10]:
wrong_index={}
def get_accuracy(y,y_pred,s):
  dict={}
  length=len(y)
  new_length=int(length/5)
  acc=0.0
  single_label_acc=0.0
  for i in range(new_length):
    index=5*i
    if y[index]==y_pred[index] and y[index+1]==y_pred[index+1] and y[index+2]==y_pred[index+2] and y[index+3]==y_pred[index+3] and y[index+4]==y_pred[index+4]:
      acc=acc+1
  for i in range(length):
    if y[i]==y_pred[i] :
      single_label_acc=single_label_acc+1
    else:
      '''
      print("X_train_0",(s+i)/5,"X_train_1",(s+i))
      cv2_imshow(X_train_0[int((s+i)/5)])
      cv2_imshow(X_train_1[s+i])
      print("true: ",int(y[i]),"false: ",int(y_pred[i]))
      '''
      
      
      if (s+i) in wrong_index:
        wrong_index[s+i]=wrong_index[s+i]+1
      else:
        wrong_index[s+i]=1
      
      
      if int(y[i]) in dict:
        dict[int(y[i])]=dict[int(y[i])]+1
      else:
        dict[int(y[i])]=1
  print("Overall accuracy: ",acc/new_length)
  print("Single label accuracy: ",single_label_acc/length)
  print(dict)

In [11]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, 2,padding=1)
        self.norm1=nn.BatchNorm2d(10)
        #self.dropout2=nn.Dropout2d(0.00005)
        self.conv2 = nn.Conv2d(10, 16, 2,padding=1)
        
        self.norm2=nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 4 * 4, 46)  # 6*6 from image dimension4
        #self.norm3 = nn.BatchNorm1d(52)
        self.fc2 = nn.Linear(46,46)
        #self.dropout1=nn.Dropout(0.1)
        self.norm5 = nn.BatchNorm1d(46)
        self.fc3 = nn.Linear(46, 24)
        self.norm4 = nn.BatchNorm1d(24)
        self.fc4 = nn.Linear(24,11)
        #self.norm3 = nn.BatchNorm1d(30)
        #self.fc5 = nn.LogSoftmax()

    def forward(self, x):
        
        # (batch_size,1,16,16) ==> (batch_size,6,17,17)==> (batch_size,6,8,8)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
  
    
        x=self.norm1(x)
        #x=self.dropout2(x)
        # (batch_size,6,8,8) ==> (betach_size,16,9,9)==> (batch_size,16,4,4)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    
        x=self.norm2(x)
        x = x.view(-1, self.num_flat_features(x))

        x = F.relu(self.fc1(x))
        #x=self.norm3(x)
        x = F.relu(self.fc2(x))
        x=self.norm5(x)
        #x=self.dropout1(x)
        x = F.relu(self.fc3(x))
        #x = F.relu(self.fc5(x))
        x=self.norm4(x)
        #x = F.dropout(x, training=self.training)
        x = F.relu(self.fc4(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [12]:
import torch.optim as optim
net1 = Net()

#optimizer = optim.SGD(net1.parameters(), lr=0.001, momentum=0.95, nesterov=True, weight_decay=2e-2)
#scheduler=optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0003, max_lr=0.001， base_momentum=0.9, max_momentum=0.99)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.85)
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net1.parameters(), lr=0.001, weight_decay=1e-3,betas=(0.9,0.99))
#optimizer = optim.RMSprop(net1.parameters(), lr=0.001, weight_decay=1e-3,momentum=0.95)



# divide X_train into X_valid and X_train_new


a=X_train[0:282000]
b=y_train[0:282000]
c=X_train[272000:282000]
d=y_train[272000:282000]
'''
for i in range(400):
  a.append(X_train[46658:46659])
  b.y_train_new.append(y_train[46658:46659])
'''
X_train_new=a
y_train_new=b.long()
X_test_new=c
y_test_new=d.long()

batch_num=100
batch_size=int(len(y_train_new)/batch_num)
train_loss=[]
test_loss=[]

In [13]:
str(float(y_test_new[0]))

'8.0'

In [ ]:
net1.to(device)
input = X_train_new.cuda()

ptr=0
min_loss=10000000000000000
acc=0
acc_loss=0
early_stopping=0
for trial in range(5):
  if trial == 0:
    learning_rate=0.001
  elif trial==1:
    learning_rate=0.0003
  elif trial==2:
    learning_rate=0.0001
  elif trial==3:
    learning_rate=0.00005 
  for epoch in range(200000):
      # minibatch gradient descent
      loss=0.0
      testloss=0.0
      for k in range(batch_num):
        l_index=batch_size*k
        r_index=batch_size*(k+1)
        out = net1(input[l_index:r_index].float().cuda())
        avg_loss = criterion(out, y_train_new[:,0][l_index:r_index].cuda())
        
        if k >=96:
     
          #test_loss.append(float(avg_loss))
          testloss=testloss+(avg_loss/4)
        else:
          
          loss=loss+(avg_loss/(batch_num-4))
          optimizer.zero_grad()
          avg_loss.backward()
          optimizer.step() 
          #scheduler.step()  
          #learning_rate = 0.001
        for f in net1.parameters():
          
          f.data.sub_(f.grad.data * learning_rate)
      
      #_, predicted = torch.max(out, 1)
      #print(testloss)
      #print(loss)
      #test_loss[ptr]=testloss
      #train_loss[ptr]=float(loss)
      test_loss.append(float(testloss))
      train_loss.append(float(loss))
      # if loss does not decrease in 50 steps
      # early stopping
      if loss>=min_loss:
        acc_loss=acc_loss+1
        
        if acc_loss >=30:
          early_stopping=early_stopping+1
          #print('Early stopping！')
          break
     
      elif loss<min_loss:
        acc_loss=0
        min_loss=loss
        # record the best prediction
        #min_predicted=predicted
      
      if epoch%100==0:
        print('trial:',trial,'epoch',epoch,':',loss)
  
  if early_stopping==1:
    print("Early Stopping trial1!!!")
    optimizer = optim.Adam(net1.parameters(), lr=0.0003, weight_decay=1e-3,betas=(0.9,0.96))
    #optimizer = optim.RMSprop(net1.parameters(), lr=0.0003, weight_decay=1e-3,momentum=0.95)

  if early_stopping==2:
    print("Early Stopping trial2!!!")
    optimizer = optim.Adam(net1.parameters(), lr=0.0001, weight_decay=1e-3,betas=(0.9,0.94))
    #optimizer = optim.RMSprop(net1.parameters(), lr=0.0001, weight_decay=1e-3,momentum=0.92)
  if early_stopping==3:
    print("Early Stopping trial3!!!")
    optimizer = optim.Adam(net1.parameters(), lr=0.00005, weight_decay=1e-3,betas=(0.9,0.93))
    #optimizer = optim.RMSprop(net1.parameters(), lr=0.00005, weight_decay=1e-3,momentum=0.9)
  if early_stopping==4:
    print("End of raining!!!!!")
    break

trial: 0 epoch 0 : tensor(1.0294, device='cuda:0', grad_fn=<AddBackward0>)
trial: 0 epoch 100 : tensor(0.0073, device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.legend_handler import HandlerLine2D
len(test_loss)
xaxis=[i for i in range(278)]
line1, =plt.plot(xaxis, test_loss,label='testing(single label)')
line2, =plt.plot(xaxis, train_loss,label='training(single label)')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=4)})
plt.title('cross entropy loss vs. epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
id=[1410*i for i in range(201)]
wrong_index={}
for i in range (len(id)-1):
  out = net1(X_train[id[i]:id[i+1]].float().cuda())
  y_train_new=y_train[id[i]:id[i+1]].long().cuda()
  loss = criterion(out, y_train_new[:,0].cuda())
  _, predicted = torch.max(out, 1)
  get_accuracy(y_train_new[:,0],predicted,id[i])
#print("Final trainning accuracy(single label):",(275000-len(wrong_index))/275000)

In [ ]:
wrong_index

In [ ]:
# testing dataset
out = net1(X_test[0:70000].float().cuda())
_, predicted = torch.max(out, 1)

# organize the shape of testing result
output=[]
for i in range(14000):
  tmp=''
  tmp=tmp+str(int(predicted[5*i]))
  tmp=tmp+str(int(predicted[5*i+1]))
  tmp=tmp+str(int(predicted[5*i+2]))
  tmp=tmp+str(int(predicted[5*i+3]))
  tmp=tmp+str(int(predicted[5*i+4]))
  output.append(tmp)
print(output)

In [ ]:
# convert from nparray to pandas
import pandas as pd
from google.colab import files
index_l=[i for i in range(14000)]
df=pd.DataFrame(data=output,index=index_l,columns=['Label'])
df['Id']=index_l
df=df[['Id','Label']]


df.to_csv('df_new46.csv',index=False)
files.download('df_new46.csv')